In [1]:
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, GRU, Dense
from tensorflow.keras.optimizers import Adam

file_path = '/kaggle/input/freezing-of-gait/FOG/freezing-of-gait-exploration-main/freezing-of-gait-exploration-main/sliding_window_dataset'
file_names = [
    'S01R01experiment0', 'S01R02experiment0',
    'S02R01experiment0', 'S02R02experiment0',
    'S03R01experiment0', 'S03R01experiment1', 'S03R02experiment0', 'S03R03experiment0',
    'S04R01experiment0',
    'S05R01experiment0', 'S05R01experiment1', 'S05R01experiment2', 'S05R01experiment3',
    'S06R01experiment0',
]
file_name = [
    'S07R01experiment0', 'S07R02experiment0',
    'S08R01experiment0', 'S08R01experiment1', 'S08R01experiment2', 'S08R01experiment3',
    'S09R01experiment0', 'S09R01experiment1'
]
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)
strategy = tpu_strategy

def create_model():
    # Define your model architecture
    model = Sequential()
    model.add(InputLayer((1000, 4)))
    model.add(GRU(64))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    return model

def train_model(X_train, y_train, X_test, y_test, initial_model=None):
    with strategy.scope():
        if initial_model is None:
            model = create_model()
        else:
            model = initial_model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=150, batch_size=256)

    return model

# Load and preprocess your data
initial_model = None
for name in file_names:
    X = np.load(f'{file_path}/{name}_X.npy')
    y = np.load(f'{file_path}/{name}_y.npy')
    y = np.array([[i] for i in y])
    length = X.shape[0]
    test_data_idx_start = int(length * 0.8)
    X_train = X[:test_data_idx_start]
    y_train = y[:test_data_idx_start]
    X_test = X[test_data_idx_start:]
    y_test = y[test_data_idx_start:]
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

    # Train the model within the TPU strategy scope
    with strategy.scope():
        initial_model = train_model(X_train, y_train, X_test, y_test, initial_model)

# Save the final trained model
initial_model.save('/kaggle/working/gru_model.h5')


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.
INFO:tensorflow:Initializing the TPU system: local
INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


...

200/200 [==============================] - 26s 106ms/step - loss: 0.0077 - accuracy: 0.9984 - val_loss: 0.0198 - val_accuracy: 0.9931
Epoch 2/150
200/200 [==============================] - 16s 79ms/step - loss: 0.0062 - accuracy: 0.9988 - val_loss: 0.0188 - val_accuracy: 0.9961
Epoch 3/150
200/200 [==============================] - 16s 80ms/step - loss: 0.0073 - accuracy: 0.9985 - val_loss: 0.0345 - val_accuracy: 0.9897
Epoch 4/150
200/200 [==============================] - 16s 79ms/step - loss: 0.0066 - accuracy: 0.9986 - val_loss: 0.0261 - val_accuracy: 0.9942
Epoch 5/150
200/200 [==============================] - 16s 78ms/step - loss: 0.0069 - accuracy: 0.9986 - val_loss: 0.0255 - val_accuracy: 0.9946
Epoch 6/150
200/200 [==============================] - 16s 80ms/step - loss: 0.0060 - accuracy: 0.9988 - val_loss: 0.0162 - val_accuracy: 0.9969
Epoch 7/150
200/200 [==============================] - 16s 79ms/step - loss: 0.0059 - accuracy: 0.9988 - val_loss: 0.0197 - val_accuracy: 0.9

2023-07-18 20:53:30.295648: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-07-18 20:53:30.445572: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


157/157 [==============================] - ETA: 0s - loss: 0.0143 - accuracy: 0.9959

2023-07-18 20:53:45.693878: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.
2023-07-18 20:53:45.809514: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp.


157/157 [==============================] - 21s 105ms/step - loss: 0.0143 - accuracy: 0.9959 - val_loss: 6.5830e-06 - val_accuracy: 1.0000
Epoch 2/150
157/157 [==============================] - 12s 79ms/step - loss: 0.0058 - accuracy: 0.9991 - val_loss: 6.7807e-06 - val_accuracy: 1.0000
Epoch 3/150
157/157 [==============================] - 12s 79ms/step - loss: 0.0045 - accuracy: 0.9995 - val_loss: 3.7859e-06 - val_accuracy: 1.0000
Epoch 4/150
157/157 [==============================] - 12s 79ms/step - loss: 0.0039 - accuracy: 0.9995 - val_loss: 4.2566e-06 - val_accuracy: 1.0000
Epoch 5/150
157/157 [==============================] - 12s 79ms/step - loss: 0.0039 - accuracy: 0.9994 - val_loss: 6.2140e-06 - val_accuracy: 1.0000
Epoch 6/150
157/157 [==============================] - 13s 86ms/step - loss: 0.0035 - accuracy: 0.9996 - val_loss: 2.0619e-05 - val_accuracy: 1.0000
Epoch 7/150
157/157 [==============================] - 13s 81ms/step - loss: 0.0034 - accuracy: 0.9996 - val_loss: 2.

In [2]:
for name in file_name:
    X1 = np.load(f'{file_path}/{name}_X.npy')
    length = X1.shape[0]
    y1 = np.load(f'{file_path}/{name}_y.npy')
    y1 = np.array([[i] for i in y1])
    test_data_idx_start = int(length)
    X1_test = X1[:test_data_idx_start]
    y1_test = y1[:test_data_idx_start]

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn import metrics

# Load the saved model
model = load_model('/kaggle/working/gru_model.h5')

yhat = model.predict(X1_test)# metrics# metrics
from sklearn import metrics
TN, FP, FN, TP = metrics.confusion_matrix(y1_test, np.array([0 if y < .5 else 1 for y in yhat])).ravel()


print('Sensitivity, hit rate, recall, or true positive rate: ')
TPR = TP/(TP+FN)
print(TPR)
print('Specificity or true negative rate')
TNR = TN/(TN+FP) 
print(TNR)
print('Precision or positive predictive value')
PPV = TP/(TP+FP)
print(PPV)
print('Negative predictive value')
NPV = TN/(TN+FN)
print(NPV)
print('Fall out or false positive rate')
FPR = FP/(FP+TN)
print(FPR)
print('False negative rate')
FNR = FN/(TP+FN)
print(FNR)
print('False discovery rate')
FDR = FP/(TP+FP)
print(FDR)

print('Overall accuracy')
ACC = (TP+TN)/(TP+FP+FN+TN)
print(ACC)

578/578 [==============================] - 55s 95ms/step
Sensitivity, hit rate, recall, or true positive rate: 
0.9182186234817814
Specificity or true negative rate
0.9534711964549483
Precision or positive predictive value
0.8780487804878049
Negative predictive value
0.9696560012017426
Fall out or false positive rate
0.0465288035450517
False negative rate
0.08178137651821862
False discovery rate
0.12195121951219512
Overall accuracy
0.944047619047619
